In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/preprocessing-course-json/course.json
/kaggle/input/preprocessing-course-json/__results__.html
/kaggle/input/preprocessing-course-json/__notebook__.ipynb
/kaggle/input/preprocessing-course-json/__output__.json
/kaggle/input/preprocessing-course-json/custom.css
/kaggle/input/mooccubex-entities/comment.json
/kaggle/input/mooccubex-entities/course.json
/kaggle/input/mooccubex-entities/paper.json
/kaggle/input/mooccubex-entities/reply.json
/kaggle/input/mooccubex-entities/school.json
/kaggle/input/mooccubex-entities/video.json
/kaggle/input/mooccubex-entities/other.json
/kaggle/input/mooccubex-entities/problem.json
/kaggle/input/mooccubex-entities/user.json
/kaggle/input/mooccubex-entities/concept.json
/kaggle/input/mooccubex-entities/teacher.json
/kaggle/input/mooccubex-relations/course-teacher.txt
/kaggle/input/mooccubex-relations/concept-comment.txt
/kaggle/input/mooccubex-relations/user-video.json
/kaggle/input/mooccubex-relations/course-comment.txt
/kaggle/input/mooccube

# Read file

In [2]:
import polars as pl
course = pl.read_ndjson('/kaggle/input/preprocessing-course-json/course.json')
video = pl.read_ndjson('/kaggle/input/mooccubex-entities/video.json')

In [3]:
course_resource = course.select(pl.col(['id','resource','name_trans'])).explode('resource')
course_resource = course_resource.unnest("resource")
course_video = course_resource.filter(pl.col("resource_id").str.starts_with("V"))
course_exercise = course_resource.filter(pl.col("resource_id").str.starts_with("Ex"))

In [4]:
course_video.head()

id,titles,resource_id,chapter,name_trans
str,list[str],str,str,str
"""C_584313""","[""第一课 导论与三家分晋"", ""导论"", ""导论""]","""V_849""","""1.1.1""","""introduction to ""zizhi tongjia…"
"""C_584313""","[""第一课 导论与三家分晋"", ""智伯的覆亡"", ""智伯的覆亡""]","""V_850""","""1.2.1""","""introduction to ""zizhi tongjia…"
"""C_584313""","[""第一课 导论与三家分晋"", ""智伯悲剧的反思"", ""智伯的覆亡讨论""]","""V_851""","""1.3.1""","""introduction to ""zizhi tongjia…"
"""C_584313""","[""第二课 战国前期的政治"", ""魏文侯治国"", ""魏文侯治国""]","""V_857""","""2.1.1""","""introduction to ""zizhi tongjia…"
"""C_584313""","[""第二课 战国前期的政治"", ""吴起的悲剧"", ""吴起的悲剧""]","""V_859""","""2.2.1""","""introduction to ""zizhi tongjia…"


In [5]:
course_video = course_video.rename({
    "id": "course_id",
    "titles": "titles",
    "resource_id": "video_id",
    "chapter": "chapter",
})

In [6]:
course_exercise.head()

id,titles,resource_id,chapter,name_trans
str,list[str],str,str,str
"""C_584313""","[""第一课 导论与三家分晋"", null, ""第一课 导论与三家分晋--习题""]","""Ex_856""","""1.4""","""introduction to ""zizhi tongjia…"
"""C_584313""","[""第二课 战国前期的政治"", null, ""第二课 战国前期的政治--习题""]","""Ex_866""","""2.5""","""introduction to ""zizhi tongjia…"
"""C_584313""","[""第三课 商鞅变法"", null, ""第三课 商鞅变法--习题""]","""Ex_876""","""3.6""","""introduction to ""zizhi tongjia…"
"""C_584313""","[""第四课 秦国的外交"", null, ""第四课 秦国的外交--习题""]","""Ex_882""","""4.6""","""introduction to ""zizhi tongjia…"
"""C_584313""","[""第五课 千古一帝"", null, ""第五课 千古一帝--习题""]","""Ex_890""","""5.8""","""introduction to ""zizhi tongjia…"


In [7]:
course_exercise = course_exercise.rename({
    "id": "course_id",
    "titles": "titles",
    "resource_id": "exercise_id",
    "chapter": "chapter",
})

In [8]:
user_video = pl.read_ndjson('/kaggle/input/mooccubex-relations/user-video.json')

In [9]:
user_video.head()

seq,user_id
list[struct[2]],str
"[{""V_1395633"",[{130.0,190.0,1.0,1588431144}, {220.0,250.0,1.0,1588431234}, … {655.1,692.55,1.25,1588437514}]}, {""V_1395635"",[{135.0,170.0,1.0,1588438045}]}, … {""V_6210800"",[{6.0,13.5,1.5,1601012899}]}]","""U_112"""
"[{""V_6224262"",[{286.0,506.0,2.0,1604061626}]}]","""U_150"""
"[{""V_6432959"",[{4552.1,4556.8,1.0,1598192594}]}]","""U_172"""
"[{""V_6334508"",[{4.0,109.0,1.0,1598832781}, {119.0,254.0,1.0,1598832896}]}, {""V_6334516"",[{593.0,598.0,1.0,1598834837}]}, … {""V_6254676"",[{5.0,105.0,1.0,1603849805}]}]","""U_189"""
"[{""V_1358540"",[{59.747,64.747,1.0,1582184174}, {69.795,74.795,1.0,1582184184}, … {584.751,589.751,1.0,1582186212}]}, {""V_1358542"",[{124.588,129.588,1.0,1582186455}, {194.608,199.608,1.0,1582186525}, … {579.763,584.763,1.0,1582186910}]}, … {""V_1358782"",[{350.2,395.3,1.5,1587530324}, {492.6,95.4,1.5,1587530419}]}]","""U_197"""


In [10]:
user_video = user_video.select(pl.col(['user_id','seq'])).explode('seq')
user_video = user_video.unnest("seq")
user_video = user_video.explode('segment')
user_video = user_video.unnest("segment")

In [11]:
user_video

user_id,video_id,start_point,end_point,speed,local_start_time
str,str,f64,f64,f64,i64
"""U_112""","""V_1395633""",130.0,190.0,1.0,1588431144
"""U_112""","""V_1395633""",220.0,250.0,1.0,1588431234
"""U_112""","""V_1395633""",478.8,508.8,1.0,1588437359
"""U_112""","""V_1395633""",655.1,692.55,1.25,1588437514
"""U_112""","""V_1395635""",135.0,170.0,1.0,1588438045
…,…,…,…,…,…
"""U_36053952""","""V_8244595""",428.2,443.2,1.0,1604148171
"""U_36053952""","""V_8244595""",1170.2,1180.2,1.0,1604148186
"""U_36053952""","""V_8244595""",1876.0,1881.0,1.0,1604148196


In [12]:
user_video = user_video.group_by(["user_id", "video_id"]).agg([
    pl.col("start_point").alias("start_list"),
    pl.col("end_point").alias("end_list"),
    pl.col("speed"),
    pl.col("local_start_time"),
])

In [13]:
def merge_intervals_polars(df):
    """ Gộp khoảng thời gian bị trùng và tính tổng thời lượng đã xem thực tế """
    results = []
    
    for start_list, end_list in zip(df["start_list"], df["end_list"]):
        if start_list.is_empty() or end_list.is_empty():  # Tránh lỗi danh sách rỗng
            results.append(0.0)
            continue
        
        intervals = sorted(zip(start_list, end_list))  # Sắp xếp theo start
        merged = []
        current_start, current_end = intervals[0]

        for start, end in intervals[1:]:
            if start <= current_end:  # Nếu trùng nhau, mở rộng khoảng
                current_end = max(current_end, end)
            else:  # Nếu không trùng, lưu khoảng trước đó và bắt đầu mới
                merged.append((current_start, current_end))
                current_start, current_end = start, end

        merged.append((current_start, current_end))  # Thêm khoảng cuối
        actual_watch_time = sum(end - start for start, end in merged)
        results.append(actual_watch_time)

    return results

# Tính thời lượng đã xem
user_video = user_video.with_columns([
    (pl.col("end_list") - pl.col("start_list")).list.sum().alias("total_watch_time"),  # Tổng thời gian (có trùng)
    pl.Series("actual_watch_time", merge_intervals_polars(user_video))  # Gọi hàm xử lý toàn bộ dataframe
])

In [14]:
user_video.head()
user_video = user_video[['user_id','video_id','total_watch_time','actual_watch_time']]


In [15]:
video = video.with_columns(
    pl.col("end").list.last().alias("video_duration")
)
display(video.head())
video = video[['ccid','video_duration']]

ccid,name,start,end,text,video_duration
str,str,list[f64],list[f64],list[str],f64
"""0001603F826A3DDA9C33DC59013074…","""Video""","[1.031, 7.095, … 504.247]","[4.255, 8.119, … 509.095]","[""第二个就是短助记符在生成上面。有一个规定"", ""短助记符是这样说"", … ""那么它的要求在它都会在那个命令集的定义的时候把它明确的说出来。""]",509.095
"""0003DB14A14A53279C33DC59013074…","""Video""","[11.799, 12.581, … 365.129]","[12.581, 17.38, … 367.336]","[""大家好"", ""今天我们讨论一维系统辐射传递一般近似解"", … ""多维系统下的通用问题""]",367.336
"""0004A5C6F07E369D9C33DC59013074…","""家禽的消化系统""","[19.2, 20.84, … 491.277]","[20.83, 23.39, … 492.315]","[""各位同学 大家好"", ""我是来自中国农业大学的曹静"", … ""是胰腺""]",492.315
"""00059EBD1371A6959C33DC59013074…","""1.2 计算机硬件系统""","[19.372, 190.522, … 626.853]","[23.14, 194.458, … 630.757]","[""大家好 本视频介绍计算机硬件系统"", ""总线就其类别来说 在计算机系统当中包括"", … ""待数据交换结束之后 再中断CPU善后""]",630.757
"""0005D1DC01B4EF8F9C33DC59013074…","""什么是供应链网络""","[24.33, 25.081, … 408.031]","[25.08, 26.775, … 408.75]","[""大家好"", ""今天我们学习的知识点"", … ""谢谢大家""]",408.75


In [16]:
user_video.head()

user_id,video_id,total_watch_time,actual_watch_time
str,str,f64,f64
"""U_15767679""","""V_6340530""",388.174,388.174
"""U_16097788""","""V_1395635""",59.7,59.7
"""U_33771074""","""V_7592810""",645.0,645.0
"""U_32440729""","""V_7472597""",340.0,340.0
"""U_19842780""","""V_1358727""",25.0,25.0


In [17]:
# Define your column names
column_names = ['video_id', 'ccid']

video_ccid = pd.read_csv('/kaggle/input/mooccubex-relations/video_id-ccid.txt', header=None, names=column_names, sep='\t')
video_ccid = pl.from_pandas(video_ccid)
display(video_ccid.head())
display(video_ccid.describe())

video_id,ccid
str,str
"""V_234845""","""0000363DB5B6E0869C33DC59013074…"
"""V_234876""","""0000363DB5B6E0869C33DC59013074…"
"""V_234907""","""0000363DB5B6E0869C33DC59013074…"
"""V_293392""","""0000363DB5B6E0869C33DC59013074…"
"""V_293445""","""0000363DB5B6E0869C33DC59013074…"


statistic,video_id,ccid
str,str,str
"""count""","""2798892""","""2798892"""
"""null_count""","""0""","""0"""
"""mean""",null,null
"""std""",null,null
"""min""","""V_1""","""0000363DB5B6E0869C33DC59013074…"
"""25%""",null,null
"""50%""",null,null
"""75%""",null,null
"""max""","""V_999997""","""FFFFD001230958A39C33DC59013074…"


In [18]:
video_ccid.unique()

video_id,ccid
str,str
"""V_3599048""","""F6B72F55B936F6729C33DC59013074…"
"""V_3939509""","""1E2E28C764400BD99C33DC59013074…"
"""V_1137279""","""71B02DDBCABB64229C33DC59013074…"
"""V_8196393""","""9E90771E62B504F19C33DC59013074…"
"""V_6888492""","""1AADB63D87343DAC9C33DC59013074…"
…,…
"""V_7168785""","""0D55B16A6E841CBD9C33DC59013074…"
"""V_7639583""","""DFA0BAF0D15339D89C33DC59013074…"
"""V_4515580""","""F396B59281D2A4C19C33DC59013074…"


In [19]:
# Perform an inner join on the 'video_id' column
common_videos = course_video.join(video_ccid, on='video_id', how='inner')

# Calculate the percentage of common video_id values
percentage_common = (len(common_videos) / len(video_ccid)) * 100
percentage_common

7.192882040464584

In [20]:
# Perform an inner join on the 'video_id' column
common_videos = user_video.join(video_ccid, on='video_id', how='left')

# Calculate the percentage of common video_id values
percentage_common = (len(common_videos) / len(video_ccid)) * 100
percentage_common

70.77804359725206

In [21]:
user_video

user_id,video_id,total_watch_time,actual_watch_time
str,str,f64,f64
"""U_15767679""","""V_6340530""",388.174,388.174
"""U_16097788""","""V_1395635""",59.7,59.7
"""U_33771074""","""V_7592810""",645.0,645.0
"""U_32440729""","""V_7472597""",340.0,340.0
"""U_19842780""","""V_1358727""",25.0,25.0
…,…,…,…
"""U_30211589""","""V_5558657""",245.0,245.0
"""U_30310046""","""V_5149580""",218.4,216.8
"""U_9582167""","""V_7857876""",189.9,189.9


In [22]:
video.describe()

statistic,ccid,video_duration
str,str,f64
"""count""","""59581""",59581.0
"""null_count""","""0""",0.0
"""mean""",null,652.764479
"""std""",null,4162.08718
"""min""","""0001603F826A3DDA9C33DC59013074…",1.12
"""25%""",null,367.739
"""50%""",null,545.3
"""75%""",null,759.32
"""max""","""FFFF3CC4C6F66F849C33DC59013074…",359999.0


In [23]:
common_videos.describe()

statistic,user_id,video_id,total_watch_time,actual_watch_time,ccid
str,str,str,f64,f64,str
"""count""","""1981001""","""1981001""",1.981001e6,1.981001e6,"""1980125"""
"""null_count""","""0""","""0""",0.0,0.0,"""876"""
"""mean""",null,null,244.273693,210.913333,null
"""std""",null,null,43346.664789,43345.154883,null
"""min""","""U_10001181""","""V_1000004""",-5.9312e7,-5.9312e7,"""0000363DB5B6E0869C33DC59013074…"
"""25%""",null,null,44.9,44.7,null
"""50%""",null,null,180.0,177.0,null
"""75%""",null,null,429.5,418.0,null
"""max""","""U_9999820""","""V_999993""",32670.0,10867.9,"""FFFF3CC4C6F66F849C33DC59013074…"


In [24]:
# Perform an inner join on the 'video_id' column
common_cc = video.join(common_videos, on='ccid', how='inner')

# Calculate the percentage of common video_id values
percentage_common = (len(common_cc) / len(video)) * 100
percentage_common

2263.4917171581546

In [25]:
display(course_video.head())
display(user_video.head())
display(video.head())
display(video_ccid.head())

course_id,titles,video_id,chapter,name_trans
str,list[str],str,str,str
"""C_584313""","[""第一课 导论与三家分晋"", ""导论"", ""导论""]","""V_849""","""1.1.1""","""introduction to ""zizhi tongjia…"
"""C_584313""","[""第一课 导论与三家分晋"", ""智伯的覆亡"", ""智伯的覆亡""]","""V_850""","""1.2.1""","""introduction to ""zizhi tongjia…"
"""C_584313""","[""第一课 导论与三家分晋"", ""智伯悲剧的反思"", ""智伯的覆亡讨论""]","""V_851""","""1.3.1""","""introduction to ""zizhi tongjia…"
"""C_584313""","[""第二课 战国前期的政治"", ""魏文侯治国"", ""魏文侯治国""]","""V_857""","""2.1.1""","""introduction to ""zizhi tongjia…"
"""C_584313""","[""第二课 战国前期的政治"", ""吴起的悲剧"", ""吴起的悲剧""]","""V_859""","""2.2.1""","""introduction to ""zizhi tongjia…"


user_id,video_id,total_watch_time,actual_watch_time
str,str,f64,f64
"""U_15767679""","""V_6340530""",388.174,388.174
"""U_16097788""","""V_1395635""",59.7,59.7
"""U_33771074""","""V_7592810""",645.0,645.0
"""U_32440729""","""V_7472597""",340.0,340.0
"""U_19842780""","""V_1358727""",25.0,25.0


ccid,video_duration
str,f64
"""0001603F826A3DDA9C33DC59013074…",509.095
"""0003DB14A14A53279C33DC59013074…",367.336
"""0004A5C6F07E369D9C33DC59013074…",492.315
"""00059EBD1371A6959C33DC59013074…",630.757
"""0005D1DC01B4EF8F9C33DC59013074…",408.75


video_id,ccid
str,str
"""V_234845""","""0000363DB5B6E0869C33DC59013074…"
"""V_234876""","""0000363DB5B6E0869C33DC59013074…"
"""V_234907""","""0000363DB5B6E0869C33DC59013074…"
"""V_293392""","""0000363DB5B6E0869C33DC59013074…"
"""V_293445""","""0000363DB5B6E0869C33DC59013074…"


In [26]:
video = video_ccid.join(video,on="ccid",how="inner")

In [27]:
course_video = course_video.join(video,on="video_id",how="inner")

In [28]:
display(video.describe())
display(course_video.describe())

statistic,video_id,ccid,video_duration
str,str,str,f64
"""count""","""1316090""","""1316090""",1.31609e6
"""null_count""","""0""","""0""",0.0
"""mean""",null,null,603.448667
"""std""",null,null,3077.32539
"""min""","""V_1""","""0001603F826A3DDA9C33DC59013074…",1.12
"""25%""",null,null,336.76
"""50%""",null,null,516.637
"""75%""",null,null,742.57
"""max""","""V_999997""","""FFFF3CC4C6F66F849C33DC59013074…",359999.0


statistic,course_id,titles,video_id,chapter,name_trans,ccid,video_duration
str,str,f64,str,str,str,str,f64
"""count""","""85787""",85787.0,"""85787""","""85787""","""85787""","""85787""",85787.0
"""null_count""","""0""",0.0,"""0""","""0""","""0""","""0""",0.0
"""mean""",null,null,null,null,null,null,623.343358
"""std""",null,null,null,null,null,null,3474.963701
"""min""","""C_1025064""",null,"""V_1000000""","""0""","""""chinese style"" clothing and c…","""0001603F826A3DDA9C33DC59013074…",1.12
"""25%""",null,null,null,null,null,null,346.174
"""50%""",null,null,null,null,null,null,528.4
"""75%""",null,null,null,null,null,null,751.426
"""max""","""C_956129""",null,"""V_999997""","""9.9.9""","""“past and present” of urban de…","""FFFF3CC4C6F66F849C33DC59013074…",359999.0


In [29]:
course_video.head()

course_id,titles,video_id,chapter,name_trans,ccid,video_duration
str,list[str],str,str,str,str,f64
"""C_677105""","[""第四章：可程控仪器标准命令"", ""4.1：SCPI的目标及主要内容"", ""Video""]","""V_93474""","""4.1.3""","""test system integration techno…","""0001603F826A3DDA9C33DC59013074…",509.095
"""C_696877""","[""第五章 一维系统辐射传递一般近似解"", ""第1节 一般近似解的意义"", ""Video""]","""V_472954""","""5.1.1""","""dielectric radiation heat tran…","""0003DB14A14A53279C33DC59013074…",367.336
"""C_696659""","[""第十四章 家禽的解剖特征"", ""第二节 消化系统"", ""家禽的消化系统""]","""V_448757""","""13.2""","""animal anatomy""","""0004A5C6F07E369D9C33DC59013074…",492.315
"""C_1784725""","[""Chapter 14 The Anatomical Characteristics Of Avian"", ""Section 2 Digestive System"", ""Digestive System of Avian-Video""]","""V_5080653""","""13.2""","""animal anatomy""","""0004A5C6F07E369D9C33DC59013074…",492.315
"""C_697721""","[""1 计算机操作系统概述"", ""1.2 计算机硬件系统"", ""1.2 计算机硬件系统""]","""V_555489""","""1.2.1""","""computer operating system""","""00059EBD1371A6959C33DC59013074…",630.757


In [30]:
course_video = course_video[['course_id','video_id']]
user_video.write_csv("user_video.csv")
course_video.write_csv("course_video.csv")
video.write_csv("video.csv")